In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})

import numpy as np
import itertools
import glob

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_density_files import CUBE
import alchemy_tools as at


In [2]:
# linear fit through origin
def linear_m(x, y):
    xy_av = np.dot(x,y)
    x2_av = np.dot(x,x)
    return(xy_av/x2_av)


# Import data into cube objects

In [3]:
paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/*.cube')
paths.sort()

cube_obj = []
for p in paths:
    cube_obj.append(CUBE(p))

In [4]:
# uniform electron gas
path_ueg = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ve_00.cube'
cube_obj.insert(0, CUBE(path_ueg))

# FFT 1D-projected density

In [52]:
# fft of all densities
# store coefficients for all lambda-values

# fourier coefficients for every lambda value
fft_freqs = np.empty((len(cube_obj), cube_obj[0].data_scaled.shape[0]), dtype='complex128')

# density before scaling
dens_unscaled = np.zeros((len(cube_obj), len(cube_obj[0].data_scaled)))

lam_val = np.array([0, 4, 6, 8, 10, 12, 14, 15, 16, 18, 20, 22, 23, 24, 26, 28, 29, 30, 32, 34, 36, 38])/38
for i in range(len(cube_obj)):
    ve_ex = cube_obj[i]
    pr = ve_ex.project((1,2))
    dens_unscaled[i] = pr

    # fft for one specific lambda value
    fft = np.fft.fft(pr)
    fft_freqs[i] = fft

In [53]:
# rescaling of frequencies

fft_freqs_scaled = fft_freqs.copy()

# do not rescale the coefficients at lambda = 1
end_idx = len(lam_val)-1

scaled_freqs = range(1,5)

for f in scaled_freqs:
    # scale imaginary part
    m_imag = linear_m(lam_val[:end_idx], fft_freqs.imag[:end_idx,f])
    fft_freqs_scaled.imag[:end_idx,f] = lam_val[:end_idx]*m_imag
    # scale real part
    m_real = linear_m(lam_val[:end_idx], fft_freqs.real[:end_idx,f])
    fft_freqs_scaled.real[:end_idx,f] = lam_val[:end_idx]*m_real
    

In [54]:
# plot frequencies before and after rescaling

fig_coeff, ax_coeff = plt.subplots(1,3)
plt.rcParams.update({'font.size': 18})
fig_coeff.suptitle(r'Fourier coefficients of the 5 lowest frequencies as a function of $\lambda$' + '\n' + 'for the 1D-projected densities of dsgdb9nsd_001212')
plt.rcParams.update({'font.size': 22})

# same colors for fit and original data
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

for i in range(0,5):
    # original values
    ax_coeff[0].plot(lam_val, fft_freqs.real[:,i], '-o', label = r'$\nu = ${}'.format(i), color=colors[i])
    ax_coeff[1].plot(lam_val, fft_freqs.imag[:,i], '-o', label = r'$\nu = ${}'.format(i), color=colors[i])
    ax_coeff[2].plot(lam_val, np.abs(fft_freqs[:,i]), '-o', label = r'$\nu = ${}'.format(i), color=colors[i])
    # after rescaling
    ax_coeff[0].plot(lam_val, fft_freqs_scaled.real[:,i], '--', color=colors[i])
    ax_coeff[1].plot(lam_val, fft_freqs_scaled.imag[:,i], '--', color=colors[i])
    ax_coeff[2].plot(lam_val, np.abs(fft_freqs_scaled[:,i]), '--', color=colors[i])

ax_coeff[0].set_xlabel(r'$\lambda$')
ax_coeff[0].set_ylabel(r'$c_{\nu, real}(\lambda)$')
ax_coeff[0].legend()

ax_coeff[1].set_xlabel(r'$\lambda$')
ax_coeff[1].set_ylabel(r'$c_{\nu, im}(\lambda)$')
ax_coeff[1].legend()

ax_coeff[2].set_xlabel(r'$\lambda$')
ax_coeff[2].set_ylabel(r'$|c_{\nu}(\lambda)|$')

Text(0, 0.5, '$|c_{\\nu}(\\lambda)|$')

In [55]:
# inverse Fourier transformation of rescaled frequencies
rescaled_dens = np.zeros((len(cube_obj), len(cube_obj[0].data_scaled)))

for i in range(len(cube_obj)):

    ve_ex = cube_obj[i]
    pr = ve_ex.project((1,2))
    
    ifft = np.fft.ifft(fft_freqs_scaled[i])
    if not np.allclose(np.abs(ifft), np.abs(ifft.real)):
        print(max(np.abs(ifft) - np.abs(ifft.real)))
    rescaled_dens[i] = ifft.real

4.360595673352785e-06
3.7755281011520037e-05
6.416373028950151e-05
1.7859890254379174e-05
2.0778774865143346e-05
1.685419595831572e-05
4.1873553230625804e-05
3.563925757449349e-05
3.144131658147642e-05
6.229712086479322e-05
5.480544719273228e-05
5.5948641548980715e-05
4.309352345907569e-05
3.087351369801539e-05
1.8205497506279533e-05
5.4883729209892484e-05
0.00015518850438134973
0.00010866984782700179
5.597431262313948e-05
0.00020802540353079095
0.0016909305290532563


In [56]:
# plot density after and before rescaling

# same colors for fit and original data
from matplotlib import cm
colors = cm.jet(np.linspace(0, 1, len(cube_obj)))

x = np.linspace(0, 20, cube_obj[0].data_scaled.shape[0])

fig, ax = plt.subplots(1,1)

for i in range(len(cube_obj)):

    ve_ex = cube_obj[i]
    pr = ve_ex.project((1,2))
    
    ifft = np.fft.ifft(fft_freqs_scaled[i])
    
    ax.plot(x, rescaled_dens[i], '--', color = colors[i])
    ax.plot(x, pr, color = colors[i])
ax.set_xlabel('Cell coordinate (Ang)')
ax.set_ylabel('1D-Projected Density')

Text(0, 0.5, '1D-Projected Density')

In [57]:
for i in range(len(cube_obj)):

    ve_ex = cube_obj[i]
    pr = ve_ex.project((1,2))
    
    ifft = np.fft.ifft(fft_freqs_scaled[i])
    print(max(ifft.imag))
    
#     print(ifft.sum())

0.0013757511876194212
0.004444465792400307
0.006117986104832541
0.0033757515628082212
0.0013847371993376734
0.0025806585663260355
0.003598395289623391
0.0032604055687315485
0.0026063601494505286
0.0034061004629144863
0.0034185816944035975
0.0021835124955959205
0.0016574568420829818
0.0013572515923114237
0.0023394075257563473
0.0023815929659731683
0.003914841739815279
0.0017744985659323542
0.0016853447514110277
0.0023108850749538772
0.0035316723525419215
1.4862737907957555e-16


In [58]:
np.fft.ifft(fft_freqs_scaled[2])

array([0.18441552-8.02565711e-04j, 0.18455996-6.68786176e-04j,
       0.18467556-5.13650101e-04j, 0.18475917-3.42231248e-04j,
       0.18480835-1.59961959e-04j, 0.18482135+2.75011087e-05j,
       0.18479719+2.14415535e-04j, 0.18473565+3.95092849e-04j,
       0.18463727+5.64036270e-04j, 0.18450335+7.16073399e-04j,
       0.18433583+8.46480895e-04j, 0.18413735+9.51098351e-04j,
       0.18391109+1.02642885e-03j, 0.18366074+1.06972391e-03j,
       0.18339033+1.07905102e-03j, 0.18310426+1.05334216e-03j,
       0.18280709+9.92422217e-04j, 0.18250346+8.97016751e-04j,
       0.18219798+7.68738795e-04j, 0.18189518+6.10055016e-04j,
       0.18159933+4.24231960e-04j, 0.18131444+2.15263505e-04j,
       0.1810441 -1.22188585e-05j, 0.18079153-2.53050996e-04j,
       0.18055938-5.01654037e-04j, 0.18034993-7.52162637e-04j,
       0.18016483-9.98560767e-04j, 0.18000536-1.23482204e-03j,
       0.1798723 -1.45505148e-03j, 0.17976607-1.65362550e-03j,
       0.17968684-1.82532711e-03j, 0.17963461-1.9654731

In [59]:
# plot density at box edges as a function of lambda after and before rescaling

x = np.linspace(0, 20, cube_obj[0].data_scaled.shape[0])

fig, ax = plt.subplots(1,1)

ax.plot(lam_val, dens_unscaled[:,0], '-o', label = 'unscaled, left')
ax.plot(lam_val, rescaled_dens[:,0], '-o', label = 'scaled, left')
ax.legend()
fig, ax = plt.subplots(1,1)
ax.plot(lam_val, dens_unscaled[:,dens_unscaled.shape[1]-1], '-o', label = 'unscaled, right')
ax.plot(lam_val, rescaled_dens[:,rescaled_dens.shape[1]-1], '-o', label = 'scaled, right')

ax.legend()
# for i in range(len(cube_obj)):

#     rescaled_dens[i][0]
    
#     ax.plot(x, ifft.real, '--', color = colors[i])
#     ax.plot(x, pr, color = colors[i])
# ax.set_xlabel('Cell coordinate (Ang)')
# ax.set_ylabel('1D-Projected Density')

# Non-linear Rescaling
The linear rescaling does not improve the behaviour of the density at the box orders sufficiently. Therefore, I use no a non-linear approach for the $\lambda = [1.0,0.8]$ and compare the trapezoidal integral over all points and only 1.0 and 0.8.

In [60]:
# cube files from 0.8 to 1.0 (5 objects)
cubes_30_38 = []
density_unscaled = np.zeros((5, cube_obj[0].data_scaled.shape[0]))

for i in range(len(cube_obj)-5, len(cube_obj)):
    cubes_30_38.append(cube_obj[i])
    
    density_unscaled[i-(len(cube_obj)-5)] = cube_obj[i].project((1,2))

In [61]:
def fft_cubes(cube_objs):
    # fourier coefficients for every lambda value
    fft_freqs = np.empty((len(cube_objs), cube_objs[0].data_scaled.shape[0]), dtype='complex128')
    # fft for all cubes
    for i in range(len(cube_objs)):
        ve_ex = cube_objs[i]
        pr = ve_ex.project((1,2))
        # fft for one specific lambda value
        fft = np.fft.fft(pr)
        fft_freqs[i] = fft
    return(fft_freqs)

In [62]:
# Fourier spectrum of densities
fft_freqs_30_38 = fft_cubes(cubes_30_38)

In [63]:
# optimize densities

fig_m, ax_m = plt.subplots(1,2)
x = np.linspace(0, 20, 175)
pr = cubes_30_38[1].project((1,2))
# ax_m.plot(x, pr, label='full')

fft = np.fft.fft(pr)
fft_scaled = fft.copy()
fft_scaled[1] = fft_scaled[1]*1.08
fft_scaled[2] = fft_scaled[2]*0.92
ifft= np.fft.ifft(fft_scaled)

ax_m[0].plot(np.arange(len(fft))[0:25],np.abs(fft)[0:25], '-o', label='original')
ax_m[0].plot(np.arange(len(fft))[0:25],np.abs(fft_scaled)[0:25], '-o', label='rescaled')

ax_m[1].plot(x, pr.real, label="original")
ax_m[1].plot(x, ifft, label="rescaled")

ax_m[0].set_xlabel('Frequency')
ax_m[0].set_ylabel('Intensity')
ax_m[1].set_xlabel('Cell coordinate (Ang)')
ax_m[1].set_ylabel('Projected Density')

ax_m[0].legend()
ax_m[1].legend()


In [49]:
# rescale first 5 densities
rescaled_densities = np.zeros((len(cubes_30_38), cubes_30_38[0].data_scaled.shape[0]))
for i in range(len(cubes_30_38)):
    pr = cubes_30_38[i].project((1,2))
    # ax_m.plot(x, pr, label='full')

    fft = np.fft.fft(pr)
    fft_scaled = fft.copy()
    fft_scaled[1] = fft_scaled[1]*1.08
    fft_scaled[2] = fft_scaled[2]*0.92
    ifft = np.fft.ifft(fft_scaled)
    rescaled_densities[i] = ifft.real

In [50]:
# integrate over lambda
integral_rescaled = np.trapz(rescaled_densities, axis=0)
integral_unscaled = np.trapz(density_unscaled, axis=0)

In [51]:
fig_m, ax_m = plt.subplots(1,1)
x = np.linspace(0, 20, 175)
for dens in zip(density_unscaled,rescaled_densities):
#     ax_m.plot(x,dens[0])
    ax_m.plot(x,dens[1])
